# transforms

> Transform, rotate, scale, skew, translate and other transformation utilities for Tailwind CSS

In [ ]:
#| default_exp utilities.transforms

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Union, Dict, Any, List
from dataclasses import dataclass
from cjm_fasthtml_tailwind.core.base import (
    TailwindScale, TailwindValue, combine_classes, StandardUtility, 
    UtilityFactory, BaseFactory, SingleValueFactory,
    is_numeric_scale, is_fraction, is_custom_property, is_arbitrary_value,
    BaseUtility
)
from cjm_fasthtml_tailwind.builders.scales import (
    ScaledFactory, DirectionalScaledFactory, ScaleConfig, INSET_CONFIG, SimpleFactory,
    ScaledUtility, NegativeFactory, SPACING_CONFIG, FRACTIONS
)

from fasthtml.common import Div
from fasthtml.jupyter import JupyUvi, HTMX
from cjm_fasthtml_tailwind.core.testing import create_test_app, create_test_page, start_test_server
from IPython.display import display

## Backface Visibility Utilities

Control whether the back face of an element is visible when rotated:

In [ ]:
#| export
# Backface visibility utilities
BACKFACE_VALUES = {
    "hidden": "backface-hidden",
    "visible": "backface-visible"
}

backface = SimpleFactory(BACKFACE_VALUES, "Backface visibility utilities for controlling if an element's backface is visible") # The backface visibility factory

In [ ]:
#|exports
def test_transforms_backface_examples():
    """Test backface visibility utilities."""
    assert str(backface.hidden) == "backface-hidden"
    assert str(backface.visible) == "backface-visible"

# Run the test
test_transforms_backface_examples()

## Perspective Utilities

Control the perspective of 3D-transformed elements:

In [ ]:
#| export
# Perspective named scales
PERSPECTIVE_VALUES = {
    "dramatic": "perspective-dramatic",
    "near": "perspective-near", 
    "normal": "perspective-normal",
    "midrange": "perspective-midrange",
    "distant": "perspective-distant",
    "none": "perspective-none"
}

# Enhanced factory that supports both named values and custom perspectives
class PerspectiveFactory(SimpleFactory):
    """Factory for perspective with both named and custom values."""
    
    def __call__(
        self,
        value: str  # Custom perspective value (e.g., '500px' or CSS variable)
    ) -> str:  # The formatted perspective CSS class
        """Handle custom perspective values."""
        if is_custom_property(value):
            return f"perspective-({value})"
        elif is_arbitrary_value(value):
            return f"perspective-[{value}]"
        return f"perspective-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the perspective factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access named perspectives as attributes',
                'Call with custom value (e.g., "500px", "10rem")',
                'CSS custom properties (e.g., "--custom-perspective")'
            ],
            'options': {
                'named_perspectives': list(self._values.keys()),
                'named_values': {
                    'dramatic': '100px',
                    'near': '300px',
                    'normal': '500px',
                    'midrange': '800px',
                    'distant': '1200px'
                },
                'custom_usage': 'Call factory with value: perspective("500px")'
            }
        }

perspective = PerspectiveFactory(PERSPECTIVE_VALUES, "Perspective utilities for controlling an element's perspective when placed in 3D space") # The perspective factory

In [ ]:
#|exports
def test_transforms_perspective_examples():
    """Test perspective utilities."""
    # Named perspectives
    assert str(perspective.dramatic) == "perspective-dramatic"
    assert str(perspective.near) == "perspective-near"
    assert str(perspective.normal) == "perspective-normal"
    assert str(perspective.midrange) == "perspective-midrange"
    assert str(perspective.distant) == "perspective-distant"
    assert str(perspective.none) == "perspective-none"
    
    # Custom perspectives
    assert perspective("500px") == "perspective-[500px]"
    assert perspective("10rem") == "perspective-[10rem]"
    assert perspective("--custom-perspective") == "perspective-(--custom-perspective)"

# Run the test
test_transforms_perspective_examples()

## Perspective Origin Utilities

Control the perspective origin of 3D-transformed elements:

In [ ]:
#| export
# Perspective origin positions - combines fixed positions with custom value support
PERSPECTIVE_ORIGIN_VALUES = {
    "center": "perspective-origin-center",
    "top": "perspective-origin-top",
    "top-right": "perspective-origin-top-right",
    "right": "perspective-origin-right",
    "bottom-right": "perspective-origin-bottom-right",
    "bottom": "perspective-origin-bottom",
    "bottom-left": "perspective-origin-bottom-left",
    "left": "perspective-origin-left",
    "top-left": "perspective-origin-top-left"
}

# Enhanced factory that supports both fixed values and custom positions
class PerspectiveOriginFactory(SimpleFactory):
    """Factory for perspective origin with both fixed and custom values."""
    
    def __call__(
        self,
        value: str  # Custom position value (e.g., '50% 25%' or CSS variable)
    ) -> str:  # The formatted perspective origin CSS class
        """Handle custom position values like '50% 25%'."""
        if is_custom_property(value):
            return f"perspective-origin-({value})"
        elif is_arbitrary_value(value) or " " in value:
            return f"perspective-origin-[{value}]"
        return f"perspective-origin-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the perspective origin factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access fixed positions as attributes',
                'Call with custom position (e.g., "50% 25%")',
                'CSS custom properties (e.g., "--custom-origin")'
            ],
            'options': {
                'fixed_positions': list(self._values.keys()),
                'custom_usage': 'Call factory with position string: perspective_origin("50% 25%")'
            }
        }

perspective_origin = PerspectiveOriginFactory(
    PERSPECTIVE_ORIGIN_VALUES, 
    "Perspective origin utilities for controlling an element's perspective origin when placed in 3D space"
) # The perspective origin factory

In [ ]:
#|exports
def test_transforms_perspective_origin_examples():
    """Test perspective origin utilities."""
    # Fixed positions
    assert str(perspective_origin.center) == "perspective-origin-center"
    assert str(perspective_origin.top) == "perspective-origin-top"
    assert str(perspective_origin.bottom_right) == "perspective-origin-bottom-right"
    assert str(perspective_origin.top_left) == "perspective-origin-top-left"
    
    # Custom positions
    assert perspective_origin("50% 25%") == "perspective-origin-[50% 25%]"
    assert perspective_origin("10px 20px") == "perspective-origin-[10px 20px]"
    assert perspective_origin("--custom-origin") == "perspective-origin-(--custom-origin)"

# Run the test
test_transforms_perspective_origin_examples()

## Rotate Utilities

Control the rotation of elements with support for 3D rotations:

In [ ]:
#| export
# Rotate angle values
ROTATE_ANGLES = list(range(360)) # Common rotation angles in degrees

# Configuration for rotate utilities
ROTATE_CONFIG = ScaleConfig(
    numeric=False,  # Not using standard numeric scale
    decimals=False,
    fractions=False,
    named=None,
    special={"none": "none"},
    negative=True  # Support negative rotations
)

# Custom utility for rotate that handles angles
class RotateUtility(StandardUtility):
    """Utility class for rotation with angle support."""
    
    def _format_value(self, value: TailwindValue) -> str:
        """Format rotation value."""
        if value == "none":
            return "none"
        elif is_custom_property(str(value)):
            return f"({value})"
        elif is_arbitrary_value(str(value)):
            return f"[{value}]"
        elif isinstance(value, (int, float)) and value in ROTATE_ANGLES:
            return str(value)
        else:
            # Treat as arbitrary value if not in predefined angles
            return f"[{value}]"

# Factory for rotate with directional support
class RotateFactory(BaseFactory):
    """Factory for rotation utilities with directional support."""
    
    def __init__(self):
        """Initialize with directional sub-factories."""
        super().__init__("Rotate utilities for rotating elements")
        self.x = ScaledFactory("rotate-x", ROTATE_CONFIG, "Rotate around X axis")
        self.y = ScaledFactory("rotate-y", ROTATE_CONFIG, "Rotate around Y axis")
        self.z = ScaledFactory("rotate-z", ROTATE_CONFIG, "Rotate around Z axis")
    
    def __call__(self, value: TailwindValue, negative: bool = False) -> RotateUtility:
        """Create a rotate utility for all axes."""
        instance = RotateUtility("rotate" if not negative else "-rotate")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> RotateUtility:
        """Handle special values and angles."""
        if name == "none":
            instance = RotateUtility("rotate")
            instance._value = "none"
            return instance
        elif name.startswith("_"):
            # Handle numeric values with underscore prefix
            angle = name[1:]
            if angle.isdigit() and int(angle) in ROTATE_ANGLES:
                instance = RotateUtility("rotate")
                instance._value = angle
                return instance
        raise AttributeError(f"'RotateFactory' object has no attribute '{name}'")
    
    @property
    def negative(self) -> 'NegativeRotateFactory':
        """Return a negative variant factory."""
        return NegativeRotateFactory()
    
    def get_info(self) -> Dict[str, Any]:
        """Get information about the rotate factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                f'Predefined angles: {", ".join(str(a) for a in ROTATE_ANGLES)}',
                'Custom angles (e.g., "45deg", "0.5turn")',
                'CSS custom properties (e.g., "--rotation-angle")',
                'Access directional rotations: x, y, z'
            ],
            'options': {
                'supports_negative': True,
                'directional_variants': {
                    'x': 'Rotate around X axis',
                    'y': 'Rotate around Y axis',
                    'z': 'Rotate around Z axis'
                }
            }
        }

# Negative rotate factory
class NegativeRotateFactory:
    """Factory for negative rotation utilities."""
    
    def __call__(self, value: TailwindValue) -> RotateUtility:
        """Create a negative rotate utility."""
        instance = RotateUtility("-rotate")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> RotateUtility:
        """Handle angles as attributes."""
        if name.startswith("_"):
            angle = name[1:]
            if angle.isdigit() and int(angle) in ROTATE_ANGLES:
                instance = RotateUtility("-rotate")
                instance._value = angle
                return instance
        raise AttributeError(f"'NegativeRotateFactory' object has no attribute '{name}'")

rotate = RotateFactory() # The rotate factory

In [ ]:
#|exports
def test_transforms_rotate_examples():
    """Test rotate utilities."""
    # Basic rotations
    assert str(rotate(0)) == "rotate-0"
    assert str(rotate(45)) == "rotate-45"
    assert str(rotate(90)) == "rotate-90"
    assert str(rotate(180)) == "rotate-180"
    assert str(rotate.none) == "rotate-none"
    assert str(rotate._45) == "rotate-45"
    
    # Negative rotations
    assert str(rotate(45, negative=True)) == "-rotate-45"
    assert str(rotate.negative(90)) == "-rotate-90"
    assert str(rotate.negative._180) == "-rotate-180"
    
    # Directional rotations
    assert str(rotate.x(45)) == "rotate-x-45"
    assert str(rotate.y(90)) == "rotate-y-90"
    assert str(rotate.z(180)) == "rotate-z-180"
    assert str(rotate.x.negative(45)) == "-rotate-x-45"
    
    # Arbitrary values
    assert str(rotate("30deg")) == "rotate-[30deg]"
    assert str(rotate("0.25turn")) == "rotate-[0.25turn]"
    assert str(rotate("--rotation")) == "rotate-(--rotation)"

# Run the test
test_transforms_rotate_examples()

## Scale Utilities

Control the scale of elements with support for directional scaling:

In [ ]:
#| export
# Scale percentage values
SCALE_VALUES = [0, 50, 75, 90, 95, 100, 105, 110, 125, 150] # Common scale percentages

# Configuration for scale utilities
SCALE_CONFIG = ScaleConfig(
    numeric=False,  # Not using standard numeric scale
    decimals=False,
    fractions=False,
    named=None,
    special={"none": "none"},
    negative=True  # Support negative scaling (flip)
)

# Custom utility for scale that handles percentages
class ScaleUtility(StandardUtility):
    """Utility class for scaling with percentage support."""
    
    def _format_value(self, value: TailwindValue) -> str:
        """Format scale value."""
        if value == "none":
            return "none"
        elif is_custom_property(str(value)):
            return f"({value})"
        elif is_arbitrary_value(str(value)):
            return f"[{value}]"
        elif isinstance(value, (int, float)) and value in SCALE_VALUES:
            return str(value)
        else:
            # Treat as arbitrary value if not in predefined scales
            return f"[{value}]"

# Factory for scale with directional support
class ScaleFactory(BaseFactory):
    """Factory for scale utilities with directional and 3D support."""
    
    def __init__(self):
        """Initialize with directional sub-factories."""
        super().__init__("Scale utilities for scaling elements")
        # Create directional scale factories with custom config
        self.x = type('ScaleX', (BaseFactory,), {
            '__init__': lambda s: BaseFactory.__init__(s, "Scale X axis"),
            '__call__': lambda s, v, negative=False: self._create_directional("scale-x", v, negative),
            '__getattr__': lambda s, n: self._get_directional_attr("scale-x", n),
            'negative': property(lambda s: type('NegativeScaleX', (), {
                '__call__': lambda ns, v: self._create_directional("scale-x", v, True),
                '__getattr__': lambda ns, n: self._get_directional_attr("scale-x", n, True)
            })()),
            'get_info': lambda s: {'description': 'Scale X axis', 'valid_inputs': [], 'options': {}}
        })()
        
        self.y = type('ScaleY', (BaseFactory,), {
            '__init__': lambda s: BaseFactory.__init__(s, "Scale Y axis"),
            '__call__': lambda s, v, negative=False: self._create_directional("scale-y", v, negative),
            '__getattr__': lambda s, n: self._get_directional_attr("scale-y", n),
            'negative': property(lambda s: type('NegativeScaleY', (), {
                '__call__': lambda ns, v: self._create_directional("scale-y", v, True),
                '__getattr__': lambda ns, n: self._get_directional_attr("scale-y", n, True)
            })()),
            'get_info': lambda s: {'description': 'Scale Y axis', 'valid_inputs': [], 'options': {}}
        })()
        
        self.z = type('ScaleZ', (BaseFactory,), {
            '__init__': lambda s: BaseFactory.__init__(s, "Scale Z axis"),
            '__call__': lambda s, v, negative=False: self._create_directional("scale-z", v, negative),
            '__getattr__': lambda s, n: self._get_directional_attr("scale-z", n),
            'negative': property(lambda s: type('NegativeScaleZ', (), {
                '__call__': lambda ns, v: self._create_directional("scale-z", v, True),
                '__getattr__': lambda ns, n: self._get_directional_attr("scale-z", n, True)
            })()),
            'get_info': lambda s: {'description': 'Scale Z axis', 'valid_inputs': [], 'options': {}}
        })()
        
        # Special 3D scale utility
        self._3d = SingleValueFactory("scale-3d", "Apply 3D scaling using CSS variables")
    
    def _create_directional(self, prefix: str, value: TailwindValue, negative: bool = False) -> ScaleUtility:
        """Create a directional scale utility."""
        full_prefix = f"-{prefix}" if negative else prefix
        instance = ScaleUtility(full_prefix)
        instance._value = instance._format_value(value)
        return instance
    
    def _get_directional_attr(self, prefix: str, name: str, negative: bool = False) -> ScaleUtility:
        """Get directional scale by attribute name."""
        if name.startswith("_"):
            scale = name[1:]
            if scale.isdigit() and int(scale) in SCALE_VALUES:
                return self._create_directional(prefix, int(scale), negative)
        raise AttributeError(f"No attribute '{name}'")
    
    def __call__(self, value: TailwindValue, negative: bool = False) -> ScaleUtility:
        """Create a scale utility for all axes."""
        instance = ScaleUtility("scale" if not negative else "-scale")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> Union[ScaleUtility, SingleValueFactory]:
        """Handle special values and scale percentages."""
        if name == "_3d":
            return self._3d
        elif name == "none":
            instance = ScaleUtility("scale")
            instance._value = "none"
            return instance
        elif name.startswith("_"):
            # Handle numeric values with underscore prefix
            scale = name[1:]
            if scale.isdigit() and int(scale) in SCALE_VALUES:
                instance = ScaleUtility("scale")
                instance._value = scale
                return instance
        raise AttributeError(f"'ScaleFactory' object has no attribute '{name}'")
    
    @property
    def negative(self) -> 'NegativeScaleFactory':
        """Return a negative variant factory."""
        return NegativeScaleFactory()
    
    def get_info(self) -> Dict[str, Any]:
        """Get information about the scale factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                f'Predefined scales: {", ".join(str(v) for v in SCALE_VALUES)}%',
                'Custom scales (e.g., "1.5", "200%")',
                'CSS custom properties (e.g., "--scale-factor")',
                'Access directional scaling: x, y, z',
                'Special: _3d for 3D scaling'
            ],
            'options': {
                'supports_negative': True,
                'directional_variants': {
                    'x': 'Scale X axis',
                    'y': 'Scale Y axis',
                    'z': 'Scale Z axis',
                    '_3d': 'Apply 3D scaling'
                }
            }
        }

# Negative scale factory
class NegativeScaleFactory:
    """Factory for negative scale utilities."""
    
    def __call__(self, value: TailwindValue) -> ScaleUtility:
        """Create a negative scale utility."""
        instance = ScaleUtility("-scale")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> ScaleUtility:
        """Handle scale values as attributes."""
        if name.startswith("_"):
            scale = name[1:]
            if scale.isdigit() and int(scale) in SCALE_VALUES:
                instance = ScaleUtility("-scale")
                instance._value = scale
                return instance
        raise AttributeError(f"'NegativeScaleFactory' object has no attribute '{name}'")

scale_tw = ScaleFactory() # The scale factory (renamed to avoid conflict with built-in scale)

In [ ]:
#|exports
def test_transforms_scale_examples():
    """Test scale utilities."""
    # Basic scaling
    assert str(scale_tw(0)) == "scale-0"
    assert str(scale_tw(50)) == "scale-50"
    assert str(scale_tw(100)) == "scale-100"
    assert str(scale_tw(150)) == "scale-150"
    assert str(scale_tw.none) == "scale-none"
    assert str(scale_tw._75) == "scale-75"
    
    # Negative scaling
    assert str(scale_tw(100, negative=True)) == "-scale-100"
    assert str(scale_tw.negative(50)) == "-scale-50"
    assert str(scale_tw.negative._75) == "-scale-75"
    
    # Directional scaling
    assert str(scale_tw.x(50)) == "scale-x-50"
    assert str(scale_tw.y(150)) == "scale-y-150"
    assert str(scale_tw.z(75)) == "scale-z-75"
    assert str(scale_tw.x.negative(100)) == "-scale-x-100"
    
    # 3D scaling
    assert str(scale_tw._3d) == "scale-3d"
    
    # Arbitrary values
    assert str(scale_tw("1.5")) == "scale-[1.5]"
    assert str(scale_tw("200%")) == "scale-[200%]"
    assert str(scale_tw("--scale-custom")) == "scale-(--scale-custom)"

# Run the test
test_transforms_scale_examples()

## Skew Utilities

Control the skew transformation of elements:

In [ ]:
#| export
# Skew angle values
SKEW_ANGLES = [0, 1, 2, 3, 6, 12] # Common skew angles in degrees

# Custom utility for skew that handles angles
class SkewUtility(StandardUtility):
    """Utility class for skewing with angle support."""
    
    def _format_value(self, value: TailwindValue) -> str:
        """Format skew value."""
        if is_custom_property(str(value)):
            return f"({value})"
        elif is_arbitrary_value(str(value)):
            return f"[{value}]"
        elif isinstance(value, (int, float)) and value in SKEW_ANGLES:
            return str(value)
        else:
            # Treat as arbitrary value if not in predefined angles
            return f"[{value}]"

# Factory for skew with directional support
class SkewFactory(BaseFactory):
    """Factory for skew utilities with directional support."""
    
    def __init__(self):
        """Initialize with directional sub-factories."""
        super().__init__("Skew utilities for skewing elements with transform")
        # Create directional skew factories
        self.x = type('SkewX', (BaseFactory,), {
            '__init__': lambda s: BaseFactory.__init__(s, "Skew X axis"),
            '__call__': lambda s, v, negative=False: self._create_directional("skew-x", v, negative),
            '__getattr__': lambda s, n: self._get_directional_attr("skew-x", n),
            'negative': property(lambda s: type('NegativeSkewX', (), {
                '__call__': lambda ns, v: self._create_directional("skew-x", v, True),
                '__getattr__': lambda ns, n: self._get_directional_attr("skew-x", n, True)
            })()),
            'get_info': lambda s: {'description': 'Skew X axis', 'valid_inputs': [], 'options': {}}
        })()
        
        self.y = type('SkewY', (BaseFactory,), {
            '__init__': lambda s: BaseFactory.__init__(s, "Skew Y axis"),
            '__call__': lambda s, v, negative=False: self._create_directional("skew-y", v, negative),
            '__getattr__': lambda s, n: self._get_directional_attr("skew-y", n),
            'negative': property(lambda s: type('NegativeSkewY', (), {
                '__call__': lambda ns, v: self._create_directional("skew-y", v, True),
                '__getattr__': lambda ns, n: self._get_directional_attr("skew-y", n, True)
            })()),
            'get_info': lambda s: {'description': 'Skew Y axis', 'valid_inputs': [], 'options': {}}
        })()
    
    def _create_directional(self, prefix: str, value: TailwindValue, negative: bool = False) -> SkewUtility:
        """Create a directional skew utility."""
        full_prefix = f"-{prefix}" if negative else prefix
        instance = SkewUtility(full_prefix)
        instance._value = instance._format_value(value)
        return instance
    
    def _get_directional_attr(self, prefix: str, name: str, negative: bool = False) -> SkewUtility:
        """Get directional skew by attribute name."""
        if name.startswith("_"):
            angle = name[1:]
            if angle.isdigit() and int(angle) in SKEW_ANGLES:
                return self._create_directional(prefix, int(angle), negative)
        raise AttributeError(f"No attribute '{name}'")
    
    def __call__(self, value: TailwindValue, negative: bool = False) -> SkewUtility:
        """Create a skew utility for both axes."""
        instance = SkewUtility("skew" if not negative else "-skew")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> SkewUtility:
        """Handle angles as attributes."""
        if name.startswith("_"):
            # Handle numeric values with underscore prefix
            angle = name[1:]
            if angle.isdigit() and int(angle) in SKEW_ANGLES:
                instance = SkewUtility("skew")
                instance._value = angle
                return instance
        raise AttributeError(f"'SkewFactory' object has no attribute '{name}'")
    
    @property
    def negative(self) -> 'NegativeSkewFactory':
        """Return a negative variant factory."""
        return NegativeSkewFactory()
    
    def get_info(self) -> Dict[str, Any]:
        """Get information about the skew factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                f'Predefined angles: {", ".join(str(a) for a in SKEW_ANGLES)}',
                'Custom angles (e.g., "15deg")',
                'CSS custom properties (e.g., "--skew-angle")',
                'Access directional skewing: x, y'
            ],
            'options': {
                'supports_negative': True,
                'directional_variants': {
                    'x': 'Skew X axis',
                    'y': 'Skew Y axis'
                }
            }
        }

# Negative skew factory
class NegativeSkewFactory:
    """Factory for negative skew utilities."""
    
    def __call__(self, value: TailwindValue) -> SkewUtility:
        """Create a negative skew utility."""
        instance = SkewUtility("-skew")
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(self, name: str) -> SkewUtility:
        """Handle angles as attributes."""
        if name.startswith("_"):
            angle = name[1:]
            if angle.isdigit() and int(angle) in SKEW_ANGLES:
                instance = SkewUtility("-skew")
                instance._value = angle
                return instance
        raise AttributeError(f"'NegativeSkewFactory' object has no attribute '{name}'")

skew = SkewFactory() # The skew factory

In [ ]:
#|exports
def test_transforms_skew_examples():
    """Test skew utilities."""
    # Basic skewing
    assert str(skew(0)) == "skew-0"
    assert str(skew(3)) == "skew-3"
    assert str(skew(6)) == "skew-6"
    assert str(skew(12)) == "skew-12"
    assert str(skew._3) == "skew-3"
    
    # Negative skewing
    assert str(skew(6, negative=True)) == "-skew-6"
    assert str(skew.negative(12)) == "-skew-12"
    assert str(skew.negative._3) == "-skew-3"
    
    # Directional skewing
    assert str(skew.x(3)) == "skew-x-3"
    assert str(skew.y(6)) == "skew-y-6"
    assert str(skew.x._12) == "skew-x-12"
    assert str(skew.x.negative(6)) == "-skew-x-6"
    assert str(skew.y.negative._3) == "-skew-y-3"
    
    # Arbitrary values
    assert str(skew("15deg")) == "skew-[15deg]"
    assert str(skew("0.5rad")) == "skew-[0.5rad]"
    assert str(skew("--skew-custom")) == "skew-(--skew-custom)"

# Run the test
test_transforms_skew_examples()

## Transform Utilities

Control CSS transform properties:

In [ ]:
#| export
# Transform utilities
TRANSFORM_VALUES = {
    "none": "transform-none",
    "gpu": "transform-gpu",
    "cpu": "transform-cpu"
}

# Enhanced factory that supports both special values and custom transforms
class TransformFactory(SimpleFactory):
    """Factory for transform utilities with special and custom values."""
    
    def __call__(
        self,
        value: str  # Custom transform value or CSS variable
    ) -> str:  # The formatted transform CSS class
        """Handle custom transform values."""
        if is_custom_property(value):
            return f"transform-({value})"
        elif is_arbitrary_value(value):
            return f"transform-[{value}]"
        return f"transform-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the transform factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access special values as attributes (none, gpu, cpu)',
                'Call with custom transform (e.g., "rotate(45deg) scale(1.5)")',
                'CSS custom properties (e.g., "--custom-transform")'
            ],
            'options': {
                'special_values': list(self._values.keys()),
                'custom_usage': 'Call factory with transform: transform("rotate(45deg) scale(1.5)")'
            }
        }

transform = TransformFactory(TRANSFORM_VALUES, "Transform utilities for transforming elements") # The transform factory

In [ ]:
#|exports
def test_transforms_transform_examples():
    """Test transform utilities."""
    # Special values
    assert str(transform.none) == "transform-none"
    assert str(transform.gpu) == "transform-gpu"
    assert str(transform.cpu) == "transform-cpu"
    
    # Custom transforms
    assert transform("npu") == "transform-npu"
    assert transform("--custom-value") == "transform-(--custom-value)"

# Run the test
test_transforms_transform_examples()

## Transform Origin Utilities

Control the origin point for transformations:

In [ ]:
#| export
# Transform origin positions - combines fixed positions with custom value support
TRANSFORM_ORIGIN_VALUES = {
    "center": "origin-center",
    "top": "origin-top",
    "top-right": "origin-top-right",
    "right": "origin-right",
    "bottom-right": "origin-bottom-right",
    "bottom": "origin-bottom",
    "bottom-left": "origin-bottom-left",
    "left": "origin-left",
    "top-left": "origin-top-left"
}

# Enhanced factory that supports both fixed values and custom positions
class TransformOriginFactory(SimpleFactory):
    """Factory for transform origin with both fixed and custom values."""
    
    def __call__(
        self,
        value: str  # Custom position value (e.g., '50% 25%' or CSS variable)
    ) -> str:  # The formatted transform origin CSS class
        """Handle custom position values like '50% 25%'."""
        if is_custom_property(value):
            return f"origin-({value})"
        elif is_arbitrary_value(value) or " " in value:
            return f"origin-[{value}]"
        return f"origin-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the transform origin factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access fixed positions as attributes',
                'Call with custom position (e.g., "50% 25%")',
                'CSS custom properties (e.g., "--custom-origin")'
            ],
            'options': {
                'fixed_positions': list(self._values.keys()),
                'custom_usage': 'Call factory with position string: origin("50% 25%")'
            }
        }

origin = TransformOriginFactory(
    TRANSFORM_ORIGIN_VALUES, 
    "Transform origin utilities for specifying the origin for an element's transformations"
) # The transform origin factory

In [ ]:
#|exports
def test_transforms_origin_examples():
    """Test transform origin utilities."""
    # Fixed positions
    assert str(origin.center) == "origin-center"
    assert str(origin.top) == "origin-top"
    assert str(origin.bottom_right) == "origin-bottom-right"
    assert str(origin.top_left) == "origin-top-left"
    
    # Custom positions
    assert origin("50% 25%") == "origin-[50% 25%]"
    assert origin("10px 20px") == "origin-[10px 20px]"
    assert origin("--custom-origin") == "origin-(--custom-origin)"

# Run the test
test_transforms_origin_examples()

## Transform Style Utilities

Control whether child elements are positioned in 3D space:

In [ ]:
#| export
# Transform style utilities
TRANSFORM_STYLE_VALUES = {
    "_3d": "transform-3d",
    "flat": "transform-flat"
}

transform_style = SimpleFactory(TRANSFORM_STYLE_VALUES, "Transform style utilities for controlling if an element's children are placed in 3D space") # The transform style factory

In [ ]:
#|exports
def test_transforms_style_examples():
    """Test transform style utilities."""
    assert str(transform_style._3d) == "transform-3d"
    assert str(transform_style.flat) == "transform-flat"

# Run the test
test_transforms_style_examples()

## Translate Utilities

Control the translation of elements with support for directional and 3D translations:

In [ ]:
#| export
# Configuration for translate utilities
TRANSLATE_CONFIG = ScaleConfig(
    numeric=True,  # Support numeric spacing scale
    decimals=True,  # Support decimal values
    fractions=True,  # Support fractions like 1/2, 3/4
    named=None,
    special={
        "full": "full",
        "px": "px",
        "none": "none"
    },
    negative=True  # Support negative translations
)

# Factory for translate with directional support
class TranslateFactory(BaseFactory):
    """Factory for translate utilities with directional and 3D support."""
    
    def __init__(self):
        """Initialize with directional sub-factories."""
        super().__init__("Translate utilities for translating elements")
        # Create directional translate factories
        self.x = ScaledFactory("translate-x", TRANSLATE_CONFIG, "Translate along X axis")
        self.y = ScaledFactory("translate-y", TRANSLATE_CONFIG, "Translate along Y axis")
        self.z = ScaledFactory("translate-z", TRANSLATE_CONFIG, "Translate along Z axis")
    
    def __call__(self, value: TailwindScale, negative: bool = False) -> ScaledUtility:
        """Create a translate utility for both X and Y axes."""
        return ScaledFactory("translate", TRANSLATE_CONFIG)(value, negative)
    
    def __getattr__(self, name: str) -> ScaledUtility:
        """Handle special values and numeric scales."""
        # Delegate to ScaledFactory
        return ScaledFactory("translate", TRANSLATE_CONFIG).__getattr__(name)
    
    @property
    def negative(self) -> 'NegativeFactory':
        """Return a negative variant factory."""
        return NegativeFactory("translate", TRANSLATE_CONFIG)
    
    def get_info(self) -> Dict[str, Any]:
        """Get information about the translate factory."""
        from cjm_fasthtml_tailwind.builders.scales import NUMERIC_SCALE, DECIMAL_SCALE
        return {
            'description': self._doc,
            'valid_inputs': [
                f'Numeric scales: 0-{max(NUMERIC_SCALE)}',
                f'Decimal scales: {", ".join(str(d) for d in DECIMAL_SCALE)}',
                f'Fractions: {len(FRACTIONS)} supported (e.g., 1/2, 2/3, 3/4)',
                'Special values: full, px, none',
                'Arbitrary values: Any string with CSS units',
                'CSS custom properties: Variables starting with --',
                'Access directional translations: x, y, z'
            ],
            'options': {
                'supports_negative': True,
                'directional_variants': {
                    'x': 'Translate along X axis',
                    'y': 'Translate along Y axis',
                    'z': 'Translate along Z axis'
                }
            }
        }

translate = TranslateFactory() # The translate factory

In [ ]:
#|exports
def test_transforms_translate_examples():
    """Test translate utilities."""
    # Basic translations
    assert str(translate(0)) == "translate-0"
    assert str(translate(4)) == "translate-4"
    assert str(translate(8)) == "translate-8"
    assert str(translate(2.5)) == "translate-2.5"
    assert str(translate.px) == "translate-px"
    assert str(translate.full) == "translate-full"
    assert str(translate.none) == "translate-none"
    
    # Fractions
    assert str(translate("1/2")) == "translate-1/2"
    assert str(translate("3/4")) == "translate-3/4"
    
    # Negative translations
    assert str(translate(4, negative=True)) == "-translate-4"
    assert str(translate.negative(8)) == "-translate-8"
    assert str(translate.negative.full) == "-translate-full"
    assert str(translate.negative("1/2")) == "-translate-1/2"
    
    # Directional translations
    assert str(translate.x(4)) == "translate-x-4"
    assert str(translate.y(8)) == "translate-y-8"
    assert str(translate.z(2)) == "translate-z-2"
    assert str(translate.x.full) == "translate-x-full"
    assert str(translate.y("1/2")) == "translate-y-1/2"
    assert str(translate.x.negative(4)) == "-translate-x-4"
    assert str(translate.y.negative.full) == "-translate-y-full"
    
    # Arbitrary values
    assert str(translate("10px")) == "translate-[10px]"
    assert str(translate("2.5rem")) == "translate-[2.5rem]"
    assert str(translate("--spacing-lg")) == "translate-(--spacing-lg)"

# Run the test
test_transforms_translate_examples()

## Practical Examples

Let's see how to use these transform utilities in real FastHTML components:

In [ ]:
#|exports
def test_transforms_fasthtml_examples():
    """Test transform utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, Button, Card, Img, A
    from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration, animate
    from cjm_fasthtml_tailwind.utilities.layout import position, top, left, display_tw
    from cjm_fasthtml_tailwind.utilities.sizing import h, w
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, grid_display
    
    # Hover effect with scale and rotate
    hover_button = Button(
        "Hover Me",
        cls=combine_classes(
            scale_tw(110).hover, 
            rotate(3).hover,
            transition.transform
        )
    )
    assert hover_button.attrs['class'] == "hover:scale-110 hover:rotate-3 transition-transform"
    
    # 3D card flip effect
    flip_card = Div(
        Div(
            Div("Front", cls=str(backface.hidden)),
            Div("Back", cls=combine_classes(backface.hidden, str(rotate.y(180)))),
            cls=combine_classes(transform_style._3d, transition.transform, duration(700), rotate.y(180).hover)
        ),
        cls=combine_classes(perspective.normal)
    )
    
    # Centered modal with translate
    modal = Div(
        Div(
            "Modal Content",
            cls=combine_classes(
                position.fixed,
                top("1/2"),
                left("1/2"),
                str(translate.x.negative("1/2")),
                str(translate.y.negative("1/2"))
            )
        )
    )
    
    # Skewed section divider
    skewed_divider = Div(
        cls=combine_classes(
            str(skew.y.negative(3)),
            origin.left,
            h(24),
            bg.gray._100
        )
    )
    
    # Parallax effect element
    parallax_element = Div(
        Img(src="bg.jpg"),
        cls=combine_classes(
            str(scale_tw(110)),
            str(translate.y("calc(var(--scroll-y) * -0.5)")),
            transform.gpu
        )
    )
    
    # 3D rotating cube
    cube_face = lambda side: Div(
        side,
        cls=combine_classes(
            position.absolute,
            w.full,
            h.full,
            backface.hidden if side != "Front" else ""
        )
    )
    
    cube = Div(
        cube_face("Front"),
        cube_face("Back"),
        cube_face("Top"),
        cube_face("Bottom"),
        cube_face("Left"),
        cube_face("Right"),
        cls=combine_classes(
            transform_style._3d,
            animate("spin-3d")
        )
    )
    
    # Return all examples in a grid layout
    return Div(
        hover_button,
        flip_card,
        modal,
        skewed_divider,
        parallax_element,
        cube,
        cls=combine_classes(grid_display, gap(5))
    )

# Run the test
test_transforms_fasthtml_examples()

```html
<div class="grid gap-5">
<button class="hover:scale-110 hover:rotate-3 transition-transform">Hover Me</button>  <div class="perspective-normal">
    <div class="transform-3d transition-transform duration-700 hover:rotate-y-180">
      <div class="backface-hidden">Front</div>
      <div class="backface-hidden rotate-y-180">Back</div>
    </div>
  </div>
  <div>
    <div class="fixed top-1/2 left-1/2 -translate-x-1/2 -translate-y-1/2">Modal Content</div>
  </div>
  <div class="-skew-y-3 origin-left h-24 bg-gray-100"></div>
  <div class="scale-110 translate-y-[calc(var(--scroll-y) * -0.5)] transform-gpu">
<img src="bg.jpg">  </div>
  <div class="transform-3d animate-spin-3d">
    <div class="absolute w-full h-full">Front</div>
    <div class="absolute w-full h-full backface-hidden">Back</div>
    <div class="absolute w-full h-full backface-hidden">Top</div>
    <div class="absolute w-full h-full backface-hidden">Bottom</div>
    <div class="absolute w-full h-full backface-hidden">Left</div>
    <div class="absolute w-full h-full backface-hidden">Right</div>
  </div>
</div>

```

In [ ]:
#| eval: false
#| output: false
test_func = test_transforms_fasthtml_examples
app, rt = create_test_app()

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

## Helper Functions

Convenient functions for common transform patterns:

In [ ]:
#| export
def center_transform() -> str:
    """Center an element using transform translate."""
    return combine_classes(
        str(translate.x.negative("1/2")),
        str(translate.y.negative("1/2"))
    )

In [ ]:
#| export
def hover_scale(scale: int = 110) -> str:
    """Create a hover scale effect."""
    return f"hover:scale-{scale} transition-transform"

In [ ]:
#| export
def flip_card_3d(perspective_value: str = "normal") -> Dict[str, str]:
    """Get classes for a 3D flip card effect."""
    return {
        "container": str(getattr(perspective, perspective_value)),
        "inner": combine_classes(
            transform_style._3d,
            "transition-transform duration-700",
            "hover:rotate-y-180"
        ),
        "front": str(backface.hidden),
        "back": combine_classes(backface.hidden, str(rotate.y(180)))
    }

In [ ]:
#| export
def parallax_transform(speed: float = 0.5) -> str:
    """Create a parallax transform effect."""
    return combine_classes(
        str(translate.y(f"calc(var(--scroll-y) * -{speed})")),
        transform.gpu
    )

In [ ]:
#|exports
def test_transforms_helper_examples():
    """Test helper functions for common transform patterns."""
    # Test center transform
    assert center_transform() == "-translate-x-1/2 -translate-y-1/2"
    
    # Test hover scale
    assert hover_scale() == "hover:scale-110 transition-transform"
    assert hover_scale(125) == "hover:scale-125 transition-transform"
    
    # Test flip card 3D
    flip_classes = flip_card_3d()
    assert flip_classes["container"] == "perspective-normal"
    assert "transform-3d" in flip_classes["inner"]
    assert flip_classes["front"] == "backface-hidden"
    assert "backface-hidden" in flip_classes["back"]
    assert "rotate-y-180" in flip_classes["back"]
    
    # Test parallax transform
    assert "translate-y-[calc(var(--scroll-y) * -0.5)]" in parallax_transform()
    assert "transform-gpu" in parallax_transform()

# Run the test
test_transforms_helper_examples()

In [ ]:
#| export
def test_transforms_factory_documentation():
    """Test that transform factories have accessible documentation."""
    # Test main factory documentation
    assert backface.describe() == "Backface visibility utilities for controlling if an element's backface is visible"
    assert perspective.describe() == "Perspective utilities for controlling an element's perspective when placed in 3D space"
    assert perspective_origin.describe() == "Perspective origin utilities for controlling an element's perspective origin when placed in 3D space"
    assert rotate.describe() == "Rotate utilities for rotating elements"
    assert scale_tw.describe() == "Scale utilities for scaling elements"
    assert skew.describe() == "Skew utilities for skewing elements with transform"
    assert transform.describe() == "Transform utilities for transforming elements"
    assert origin.describe() == "Transform origin utilities for specifying the origin for an element's transformations"
    assert transform_style.describe() == "Transform style utilities for controlling if an element's children are placed in 3D space"
    assert translate.describe() == "Translate utilities for translating elements"
    
    # Test directional sub-factories
    assert rotate.x.describe() == "Rotate around X axis"
    assert rotate.y.describe() == "Rotate around Y axis"
    assert rotate.z.describe() == "Rotate around Z axis"
    assert translate.x.describe() == "Translate along X axis"
    assert translate.y.describe() == "Translate along Y axis"
    assert translate.z.describe() == "Translate along Z axis"
    
    # Test get_info methods
    perspective_info = perspective.get_info()
    assert 'named_perspectives' in perspective_info['options']
    assert 'dramatic' in perspective_info['options']['named_perspectives']
    
    rotate_info = rotate.get_info()
    assert 'supports_negative' in rotate_info['options']
    assert rotate_info['options']['supports_negative'] == True
    
    translate_info = translate.get_info()
    assert 'directional_variants' in translate_info['options']
    assert 'x' in translate_info['options']['directional_variants']

# Run the test
test_transforms_factory_documentation()

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()